In [23]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from datetime import timedelta
from statsmodels.tsa.ar_model import AutoReg

In [24]:
data = pd.read_csv("../data/store_demand.csv")
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)
data['store_item'] = data.store.astype(str) + '-' + data.item.astype(str)
data.drop(['store', 'item'], axis=1, inplace=True)


In [25]:
def train_test_splitter(df, pct=0.8):
    timespan = (df.index.max() - df.index.min()).days
    train_date = round(timespan*pct)
    train = df.index.min() + timedelta(days=train_date)
    return [df[df.index <= train], df[df.index > train]]


train, test = train_test_splitter(data, .7)


In [26]:
# train = train[train.store_item.isin(['1-1', '1-2'])]
# test = test[test.store_item.isin(['1-1', '1-2'])]
# subset for testing
 


In [27]:
preds = []
for series in train.store_item.unique():
    temp = train[train.store_item == series]
    model = AutoReg(temp.sales, lags=1, old_names=False)
    model_fit = model.fit()
    yhat = model_fit.predict(len(temp), round((len(temp)*1/0.7) -2)) # find a less ratchet way
    # to make the predictions match the testing set
    preds.append(yhat)


In [28]:
preds_flat = [x for sub in preds for x in sub]

In [29]:
dates = []
for item in preds:
    for sub in item.index:
        dates.append(sub)

In [30]:
pd.DataFrame(
    {
        'sales': test.sales, 
        'autoreg': preds_flat
    }, index = dates
)

,sales,autoreg
2016-07-03,35,24.641378
2016-07-04,18,21.440238
2016-07-05,31,20.047682
2016-07-06,22,19.441893
2016-07-07,33,19.178364
...,...,...
2017-12-27,63,70.187529
2017-12-28,59,70.187529
2017-12-29,74,70.187529
2017-12-30,62,70.187529
